In [ ]:
import gzip
import json
import pandas as pd
import os


In [ ]:
!curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
!unzip awscliv2.zip
!sudo ./aws/install


Streaming output truncated to the last 5000 lines.
  inflating: aws/dist/awscli/examples/appconfig/update-configuration-profile.rst  
  inflating: aws/dist/awscli/examples/appconfig/delete-configuration-profile.rst  
  inflating: aws/dist/awscli/examples/appconfig/delete-extension-association.rst  
  inflating: aws/dist/awscli/examples/appconfig/update-application.rst  
  inflating: aws/dist/awscli/examples/appconfig/list-extension-associations.rst  
  inflating: aws/dist/awscli/examples/iotanalytics/tag-resource.rst  
  inflating: aws/dist/awscli/examples/iotanalytics/delete-dataset.rst  
  inflating: aws/dist/awscli/examples/iotanalytics/create-pipeline.rst  
  inflating: aws/dist/awscli/examples/iotanalytics/update-dataset.rst  
  inflating: aws/dist/awscli/examples/iotanalytics/sample-channel-data.rst  
  inflating: aws/dist/awscli/examples/iotanalytics/update-datastore.rst  
  inflating: aws/dist/awscli/examples/iotanalytics/start-pipeline-reprocessing.rst  
  inflating: aws/dist/

In [ ]:
!aws --version


aws-cli/2.27.61 Python/3.13.4 Linux/6.1.123+ exe/x86_64.ubuntu.22


# Just one

In [ ]:
!aws s3 sync s3://neuronpedia-datasets/v1/gemma-2-2b/0-gemmascope-att-16k/explanations/ ./explanations --no-sign-request

download: s3://neuronpedia-datasets/v1/gemma-2-2b/0-gemmascope-att-16k/explanations/.DS_Store to explanations/.DS_Store
download: s3://neuronpedia-datasets/v1/gemma-2-2b/0-gemmascope-att-16k/explanations/batch-1.jsonl.gz to explanations/batch-1.jsonl.gz
download: s3://neuronpedia-datasets/v1/gemma-2-2b/0-gemmascope-att-16k/explanations/batch-10.jsonl.gz to explanations/batch-10.jsonl.gz
download: s3://neuronpedia-datasets/v1/gemma-2-2b/0-gemmascope-att-16k/explanations/batch-11.jsonl.gz to explanations/batch-11.jsonl.gz
download: s3://neuronpedia-datasets/v1/gemma-2-2b/0-gemmascope-att-16k/explanations/batch-0.jsonl.gz to explanations/batch-0.jsonl.gz
download: s3://neuronpedia-datasets/v1/gemma-2-2b/0-gemmascope-att-16k/explanations/batch-15.jsonl.gz to explanations/batch-15.jsonl.gz
download: s3://neuronpedia-datasets/v1/gemma-2-2b/0-gemmascope-att-16k/explanations/batch-3.jsonl.gz to explanations/batch-3.jsonl.gz
download: s3://neuronpedia-datasets/v1/gemma-2-2b/0-gemmascope-att-16k

In [ ]:
# Path to the folder with .jsonl.gz files
folder_path = "explanations"

# Store all data here
all_data = []

# Loop through sorted files
for filename in sorted(os.listdir(folder_path)):
    if filename.endswith(".jsonl.gz"):
        with gzip.open(os.path.join(folder_path, filename), 'rt', encoding='utf-8') as f:
            for line in f:
                try:
                    data = json.loads(line)
                    all_data.append(data)
                except json.JSONDecodeError:
                    continue  # skip bad lines

# Convert to DataFrame
df = pd.DataFrame(all_data)

In [ ]:
print(df.shape)
df.head()

(16273, 13)


,id,modelId,layer,index,authorId,description,embedding,typeName,explanationModelName,umap_x,umap_y,umap_cluster,umap_log_feature_sparsity
0,cm4f6xvdu005zvg721bbth9rf,gemma-2-2b,0-gemmascope-att-16k,0,clkht01d40000jv08hvalcvly,attends from sequences of dots to sequences of...,"[-0.10424283,-0.04151461,-0.02295776,0.0510668...",oai_attention-head,gemini-1.5-flash,0.0,0.0,0,0.0
1,qa4mz9ulru818d3qr58a6dksu,gemma-2-2b,0-gemmascope-att-16k,0,clkht01d40000jv08hvalcvly,attends to empty or punctuated segments from ...,"[0.0021070386,-0.053381737,-0.05095903,-0.0642...",oai_attention-head,gpt-4o-mini,0.0,0.0,0,0.0
2,co538yo4ujvhu33zqk1fsled0,gemma-2-2b,0-gemmascope-att-16k,1,clkht01d40000jv08hvalcvly,attends to characters or terms related to use...,"[-0.013107698,0.16262285,-0.05308618,0.0238997...",oai_attention-head,gpt-4o-mini,0.0,0.0,0,0.0
3,o22zi2qxodw5b5viynlj82h1e,gemma-2-2b,0-gemmascope-att-16k,10,clkht01d40000jv08hvalcvly,attends to tokens that imply movement or prog...,"[-0.078943,0.0007921398,-0.025863599,-0.071860...",oai_attention-head,gpt-4o-mini,0.0,0.0,0,0.0
4,hahucc54hq9wo2uaf0c5ub341,gemma-2-2b,0-gemmascope-att-16k,100,clkht01d40000jv08hvalcvly,attends to the token after the period in the ...,"[0.0038431592,-0.006530103,-0.0050395904,-0.02...",oai_attention-head,gpt-4o-mini,0.0,0.0,0,0.0


# Putting it all together

In [ ]:
# STEP 2: Set up loop over all layers and types
import os

layers = list(range(26))
types = ["res", "mlp", "att"]

# Make base directory
os.makedirs("all_explanations", exist_ok=True)

for typ in types:
    for layer in layers:
        s3_path = f"s3://neuronpedia-datasets/v1/gemma-2-2b/{layer}-gemmascope-{typ}-16k/explanations/"
        local_path = f"./all_explanations/{layer}-{typ}"
        os.makedirs(local_path, exist_ok=True)
        print(f"Downloading: {s3_path}")
        !aws s3 sync {s3_path} {local_path} --no-sign-request


Downloading: s3://neuronpedia-datasets/v1/gemma-2-2b/0-gemmascope-res-16k/explanations/
download: s3://neuronpedia-datasets/v1/gemma-2-2b/0-gemmascope-res-16k/explanations/batch-16.jsonl.gz to all_explanations/0-res/batch-16.jsonl.gz
download: s3://neuronpedia-datasets/v1/gemma-2-2b/0-gemmascope-res-16k/explanations/batch-12.jsonl.gz to all_explanations/0-res/batch-12.jsonl.gz
download: s3://neuronpedia-datasets/v1/gemma-2-2b/0-gemmascope-res-16k/explanations/batch-10.jsonl.gz to all_explanations/0-res/batch-10.jsonl.gz
download: s3://neuronpedia-datasets/v1/gemma-2-2b/0-gemmascope-res-16k/explanations/batch-0.jsonl.gz to all_explanations/0-res/batch-0.jsonl.gz
download: s3://neuronpedia-datasets/v1/gemma-2-2b/0-gemmascope-res-16k/explanations/batch-15.jsonl.gz to all_explanations/0-res/batch-15.jsonl.gz
download: s3://neuronpedia-datasets/v1/gemma-2-2b/0-gemmascope-res-16k/explanations/batch-1.jsonl.gz to all_explanations/0-res/batch-1.jsonl.gz
download: s3://neuronpedia-datasets/v1/g

In [ ]:
# STEP 3: Extract and Combine All .jsonl.gz Files
import gzip
import json
import pandas as pd

all_data = []

for typ in types:
    for layer in layers:
        subdir = f"all_explanations/{layer}-{typ}"
        if not os.path.exists(subdir):
            continue
        for fname in os.listdir(subdir):
            if fname.endswith(".jsonl.gz"):
                with gzip.open(os.path.join(subdir, fname), 'rt', encoding='utf-8') as f:
                    for line in f:
                        try:
                            data = json.loads(line)
                            data["layer"] = layer
                            data["type"] = typ
                            all_data.append(data)
                        except json.JSONDecodeError:
                            continue

df = pd.DataFrame(all_data)
print(f"Combined {len(df)} rows.")


Combined 1291008 rows.


In [12]:
df.shape

(1291008, 14)

In [11]:
df.head()

,id,modelId,layer,index,authorId,description,embedding,typeName,explanationModelName,umap_x,umap_y,umap_cluster,umap_log_feature_sparsity,type
0,clz2416dz4coqweox4tih83kj,gemma-2-2b,0,10915,clkht01d40000jv08hvalcvly,"references to the ""front"" aspect of objects, p...","[0.0032281638,0.076307066,-0.0338971,0.0167500...",oai_token-act-pair,gpt-4o-mini,10.474238,4.928702,1,0.0,res
1,clz2416dz4corweoxq1o8mbzh,gemma-2-2b,0,10916,clkht01d40000jv08hvalcvly,terms related to healthcare systems and relat...,"[-0.0957783,0.20428514,-0.06501767,-0.00444663...",oai_token-act-pair,gpt-4o-mini,11.165978,1.494609,1,0.0,res
2,clz2416dz4cosweoxyhne33td,gemma-2-2b,0,10917,clkht01d40000jv08hvalcvly,keywords related to centrality and adverse ef...,"[0.02316473,0.0034923926,-0.026956752,-0.03186...",oai_token-act-pair,gpt-4o-mini,11.143732,4.794233,1,0.0,res
3,clz2416dz4cotweoxmnmt88ag,gemma-2-2b,0,10918,clkht01d40000jv08hvalcvly,"instances of the word ""burst""","[0.073809154,0.07114081,-0.056804106,-0.020611...",oai_token-act-pair,gpt-4o-mini,11.926755,3.714641,1,0.0,res
4,clz2416dz4couweox6an8nk9v,gemma-2-2b,0,10919,clkht01d40000jv08hvalcvly,concepts related to transparency in systems or...,"[-0.098148,0.045437377,-0.041780323,0.02850051...",oai_token-act-pair,gpt-4o-mini,13.543831,4.360817,1,0.0,res


In [ ]:
# STEP 4: Save to CSV
df.to_csv("all_explanations_combined.csv", index=False)
# !cp all_explanations_combined.csv /content/drive/MyDrive/

In [ ]:
import os

file_path = "all_explanations_combined.csv"
size_bytes = os.path.getsize(file_path)
size_mb = size_bytes / (1024 * 1024)
size_gb = size_bytes / (1024 * 1024 * 1024)

print(f"File size: {size_mb:.2f} MB ({size_gb:.2f} GB)")


File size: 4071.44 MB (3.98 GB)


In [13]:
import duckdb

# Save to DuckDB file
con = duckdb.connect("explanations.duckdb")
con.register("my_df", df)  # Register the DataFrame as a view
con.execute("CREATE TABLE neurons AS SELECT * FROM my_df")
con.close()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
import duckdb

# Connect to your DuckDB database file
con = duckdb.connect("my_neurons.duckdb")
# Example query: find a row with type='res', layer=25, neuron_id=10023
query = """
SELECT neuron_id, description, layer, type
FROM neurons
WHERE type = 'res' AND layer = 25 AND neuron_id = 10023
"""

result_df = con.execute(query).df()
print(result_df)
